# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

In [1]:
from xml.etree import ElementTree as ET
from glob import glob
import os
from typing import List
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 発想

これまで、RNNを用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できそう。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプションなどが例に挙げられる。  
学習方法は簡単で、入力画像に対して適切な文章が出力されるように学習させるだけ。隠れ状態を通じてRNNからCNNまで逆伝播を繋げる。

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。  

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。  
こういった、シーケンスをシーケンスに変換するモデルを *seq2seq (Sequence to Sequence)* と呼ぶ。

本章では、入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。

このモデルは以下の二つのRNNから構成される。
- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットを使用する。
- [Wikipedia日英京都関連文書対訳コーパス](https://alaginrc.nict.go.jp/WikiCorpus/index.html)

In [3]:
data_ja, data_en = [], []
root_dir = 'data/jaen-kyoto/'
for xml_file in glob(os.path.join(root_dir, '*/*.xml')):
    try:
        tree = ET.parse(xml_file)
    except ET.ParseError:
        continue
    root = tree.getroot()
    for sentence in root.iter('sen'):
        ja = sentence.find('j').text
        en = sentence.findall('e')[-1].text
        if ja and en:
            data_ja.append(ja)
            data_en.append(en)

# examples
for _ in range(5):
    i = random.randint(0, len(data_en))
    print('en:', data_en[i])
    print('ja:', data_ja[i])
    print()

n_data = len(data_en)
print('num of data:', n_data)

en: Therefore, it is considered that the 'being that caused bizarre things' came to be called yokai.
ja: そのため「怪異を起こす存在」を妖怪と呼ぶようになったと考えられる。

en: Yoshitsune Senbonzakura' (Yoshitsune and One Thousand Cherry Trees), another title of a Kabuki play that has Yoshitsune in the leading role, also has Benkei as one of the characters.
ja: また、義経を主人公とした「義経千本桜」などの歌舞伎にも、主要人物の一人として登場する。

en: Misao YASHIRO (the founder of Meiji Horitsu Gakko)
ja: 矢代操（明治法律学校の創立者）

en: In 785, the involvement of his father, Tsuguhito, in the assassination of FUJIWARA no Tanetsugu also implicated his involvement, and he was also deported to Sado Province.
ja: 785年、父継人が藤原種継暗殺事件に関与したため、連座して佐渡国に流される。

en: Festival in honor of Ebisu (October 19 and 20):
ja: 恵比寿講（10月19日・20日）

num of data: 443596


書き出し

In [4]:
textfile_ja = 'data/kyoto_ja.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = 'data/kyoto_en.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

トークナイザの学習

In [5]:
pad_id = 3

vocab_size_ja = 8000
tokenizer_prefix_ja = f'models/tokenizer_kyoto_ja'
spm.SentencePieceTrainer.Train(
    input=textfile_ja, # データセット
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size_ja,
    pad_id=pad_id
)

vocab_size_en = 8000
tokenizer_prefix_en = f'models/tokenizer_kyoto_en'
spm.SentencePieceTrainer.Train(
    input=textfile_en, # データセット
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size_en,
    pad_id=pad_id
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/kyoto_ja.txt
  input_format: 
  model_prefix: models/tokenizer_kyoto_ja
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_

データ数を減らしたものも作っておく。

In [6]:
n_data = 10000
data_ja = data_ja[:n_data]
data_en = data_en[:n_data]

textfile_ja = f'data/kyoto_ja_{n_data}.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = f'data/kyoto_en_{n_data}.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

vocab_size_ja = 8000
tokenizer_prefix_ja = f'models/tokenizer_kyoto_ja_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_ja, # データセット
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size_ja,
    pad_id=pad_id
)

vocab_size_en = 8000
tokenizer_prefix_en = f'models/tokenizer_kyoto_en_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_en, # データセット
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size_en,
    pad_id=pad_id
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/kyoto_ja_10000.txt
  input_format: 
  model_prefix: models/tokenizer_kyoto_ja_10000
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  d

### 前処理

データ数を減らした方を使う。

トークナイザ読み込み

In [7]:
sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()
n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [8]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [9]:
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。  
また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Eecoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

DataLoaderの作成。

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # encoderへの入力
        x_dec = en[:-1] # decoderへの入力
        y_dec = en[1:] # decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

(torch.Size([32, 84]), torch.Size([32, 124]), torch.Size([32, 124]))


---

## モデル構築

EncoderとDecoderを作る。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN(LSTM)。LSTMと線形層で作る。

ここで1つ工夫を加える。普通に作っても動きはするが、学習が上手くいかない可能性がある。

encoderへの入力はpaddingされたデータである。ここで、paddingされた範囲が多い=padトークンが多く含まれているデータは、padの数が多くなるにつれて、隠れ状態がある一定の値に収束してしまう。RNNに同じトークンを何度も入力することで隠れ状態が収束していってしまうのだ。  
そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、翻訳を学習できない。

そこで、encoderが出力する隠れ状態として、padトークンを除いた最後のトークン=EOSを入力した時点のものを使用する。  

以上を踏まえ、encoderを以下のように実装する。

In [27]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        # self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        # hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
        hs, _ = self.rnn(x) # (batch_size, seq_len, hidden_size)
        h = hs[eos_positions] # (batch_size, hidden_size)
        h = self.fc(h).unsqueeze(0) # (1, batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [28]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        # self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, hc):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        # hs, (h, c) = self.lstm(x, hc)
        hs, h = self.rnn(x, hc)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
            # c: (1, batch_size, hidden_size)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        # return y, (h, c)
        return y, h

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        # hc = (h, torch.zeros_like(h))
        # y, _ = self.decoder(x_dec, hc)
        y, _ = self.decoder(x_dec, h)
        return y

### 学習

In [14]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

def eval_model(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x_enc, x_dec, y_dec in test_loader:
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)
            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            total_loss += loss.item()
    loss = total_loss / len(test_loader)
    return loss

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}', no_step=True)
        prog.memo()

In [30]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [31]:
train(model, optimizer, n_epochs=200, prog_unit=20)

   1-20/200: ############################## 100% [00:01:54.80] loss train: 4.15695, test: 3.99432 
  21-40/200: ############################## 100% [00:01:55.74] loss train: 2.67285, test: 3.99094 
  41-60/200: ############################## 100% [00:01:55.98] loss train: 1.92427, test: 4.28475 
  61-80/200: ############################## 100% [00:01:56.13] loss train: 1.39262, test: 4.66566 
 81-100/200: ############################## 100% [00:01:56.07] loss train: 0.99579, test: 5.06977 
101-120/200: ############################## 100% [00:01:55.67] loss train: 0.69274, test: 5.48003 
121-140/200: ############################## 100% [00:01:56.16] loss train: 0.46373, test: 5.89901 
141-160/200: ############################## 100% [00:02:00.77] loss train: 0.29749, test: 6.29669 
161-180/200: ############################## 100% [00:02:00.28] loss train: 0.18124, test: 6.70021 
181-200/200: ############################## 100% [00:02:05.66] loss train: 0.11154, test: 7.02597 



---

## 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。

In [34]:
unk_id = sp_en.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 50, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device)

    h = model.encoder(in_ids)
    next_token = bos_id

    token_ids = []
    for _ in range(max_len):
        x = torch.tensor([[next_token]], device=device)
        hc = (h, torch.zeros_like(h))
        # y, hc = model.decoder(x, hc)
        y, h = model.decoder(x, h)
        y = y[0]
        if decisive:
            next_token = y.argmax().item()
        else:
            next_token = token_sampling(y)
        token_ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データに含まれているものから。

In [35]:
n = 5
for x, t in zip(data_ja[:n], data_en[:n]):
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: 駅情報
output: Information
answer: Information

input: 三条京阪駅（さんじょうけいはんえき）は、京都市東山区にある、京都市営地下鉄東西線の鉄道駅。
output: Located in the Higashiyama Ward of Kyoto City, Sanjyo-Keihan Station is a stop on the Tozai Line, a Kyoto Municipal Subway Line.
answer: Located in the Higashiyama Ward of Kyoto City, Sanjyo-Keihan Station is a stop on the Tozai Line, a Kyoto Municipal Subway Line.

input: 駅番号はT11。
output: The station number is T11.
answer: The station number is T11.

input: 京阪電気鉄道
output: Keihan Electric Railway
answer: The Keihan Electric Railway

input: 京阪本線・京阪鴨東線（三条駅 (京都府)）
output: Keihan Main Line and Keihan Oto Line in Sanjyo Station (in Kyoto Prefecture)
answer: Keihan Main Line and Keihan Oto Line in Sanjyo Station (in Kyoto Prefecture)



含まれていないものも試してみる。

In [36]:
sentences = [
    'この駅は京都市内の中心部にあります。',
    '京都'
]

In [37]:
for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: この駅は京都市内の中心部にあります。
output: Marutamachi Station (Kyoto Prefecture) of Kitakinki Tango Railway Corporation (KTR), Osaka in Kyoto)

input: 京都
output: E Station: bound Station/Kyoto Station Hachijoed Keihan Station.

